## Deploy Zookeeper with Helm

Run the commands below to install zookeeper with helm. 

In [ ]:
helm upgrade --install zookeeper oci://registry-1.docker.io/bitnamicharts/zookeeper \
  --set image.tag=3.9.2 \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set persistence.size=1Gi \
  --set replicaCount=1
echo DONE

### View Zookeeper pod details
After deploying zookeeper, it will take a minute before the zookeeper pod is up and running. 
Run the command below until the zookeeper pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
oc get pod -l app.kubernetes.io/name=zookeeper
oc wait pod -l app.kubernetes.io/name=zookeeper --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/name=zookeeper
echo DONE

## Deploy Solr with Helm


Run the command below to deploy the default computate Solr ConfigSet as a Kubernetes ConfigMap to the cloud. These configsets will be loaded later, when the Solr pod is running to initialize the Solr ConfigSet which is the schema you can use for Solr Collections in your site. 

Run the commands below to install solr with helm. 

In [ ]:
helm upgrade --install solr oci://registry-1.docker.io/bitnamicharts/solr \
  --set image.tag=9.6.1 \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set zookeeper.enabled=false \
  --set externalZookeeper.servers={zookeeper:2181} \
  --set persistence.size=2Gi \
  --set collectionShards=1 \
  --set collectionReplicas=1 \
  --set replicaCount=1
echo DONE

### View solr pod details
After deploying solr, it will take a minute before the solr pod is up and running. 
Run the command below until the solr pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
oc get pod -l app.kubernetes.io/name=solr
oc wait pod -l app.kubernetes.io/name=solr --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/name=solr
echo DONE

## Deploy PostgreSQL with helm

Run the commands below to install PostgreSQL relational database with helm. 

In [ ]:
helm upgrade --install postgresql oci://registry-1.docker.io/bitnamicharts/postgresql \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set-json 'primary.resources={"limits": {"ephemeral-storage": "1Gi"}}' \
  --set image.tag=14.5.0 \
  --set primary.persistence.size=2Gi
echo DONE

### View PostgreSQL pod details
After deploying the Helm Chart, it will take a minute before the PostgreSQL pod is up and running. Run the command below until the PostgreSQL pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
oc get pod -l app.kubernetes.io/name=postgresql
oc wait pod -l app.kubernetes.io/name=postgresql --for=condition=Ready --timeout=2m
oc get pod -l app.kubernetes.io/name=postgresql
echo DONE

## Deploy the MongoDB NOSQL Database in the OpenShift Developer Sandbox

A MongoDB No-SQL Database is required for FIWARE IoT Agents to store smart device registration data about each smart device in the project. 

Deploy MongoDB with helm. 

In [ ]:
helm upgrade --install mongodb oci://registry-1.docker.io/bitnamicharts/mongodb \
 --set persistence.size=2Gi \
 --set nameOverride=mongodb \
 --set fullnameOverride=mongodb \
 --set replicaCount=1 \
 --set architecture=standalone \
 --set podSecurityContext.enabled=false \
 --set containerSecurityContext.enabled=false \
 --set "resources.limits.memory=1Gi" \
 --set "resources.limits.cpu=1" \
 --set "resources.requests.cpu=300m" \
 --set "resources.requests.memory=300Mi" \
 --set persistence.enabled=true \
 --set "persistence.size=8Gi" \
 --set readinessProbe.timeoutSeconds=60 \
 --set systemLogVerbosity=1 \
 --set-json 'podLabels={"marinera/platform":"fiware", "marinera/component":"core", "marinera/subcomponent":"persistence", "marinera/product":"mongodb"}' \
 --set metrics.enabled=true \
 --set serviceMonitor.enabled=true \
 --set arbiter.podSecurityContext.enabled=false \
 --set arbiter.containerSecurityContext.enabled=false \
 --set-json 'arbiter.podLabels={"marinera/platform":"fiware", "marinera/component":"core", "marinera/subcomponent":"persistence", "marinera/product":"mongodb"}' \
 --set arbiter.readinessProbe.timeoutSeconds=60 \
 --set "arbiter.resources.limits.memory=1Gi" \
 --set "arbiter.resources.limits.cpu=1" \
 --set "arbiter.resources.requests.cpu=200m" \
 --set "arbiter.resources.requests.memory=300Mi"
echo DONE

### View MongoDB pod details
After deploying MongoDB with helm, it will take a minute before the MongoDB pod is up and running. Run the command below until the MongoDB pod health checks are `READY 2/2` and `STATUS Running`. 

In [ ]:
oc get pod -l app.kubernetes.io/instance=mongodb
oc wait pod -l app.kubernetes.io/instance=mongodb --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/instance=mongodb
echo DONE

## Deploy RabbitMQ with helm. 

A message broker like RabbitMQ can receive messages from smart devices and forward them on to an IoT Agent to process the message. 

Create a RabbitMQ secret. 

In [ ]:
oc create secret generic rabbitmq \
  --from-literal rabbitmq-password="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 15)" \
  --from-literal rabbitmq-erlang-cookie="$(tr -dc 'A-Za-z0-9' < /dev/urandom | head -c 15)"
echo DONE

In [ ]:
helm upgrade --install rabbitmq oci://registry-1.docker.io/bitnamicharts/rabbitmq \
  --set "replicaCount=1" \
  --set "resources.limits.memory=1Gi" \
  --set "resources.limits.cpu=1" \
  --set "resources.requests.cpu=200m" \
  --set "resources.requests.memory=200Mi" \
  --set "auth.username=user" \
  --set "auth.existingPasswordSecret=rabbitmq" \
  --set "auth.existingErlangSecret=rabbitmq" \
  --set "extraPlugins='rabbitmq_auth_backend_ldap rabbitmq_auth_backend_http'" \
  --set "persistence.enabled=false" \
  --set podSecurityContext.enabled=false \
  --set containerSecurityContext.enabled=false
echo DONE

### View RabbitMQ pod details
After deploying RabbitMQ with helm, it will take a minute before the RabbitMQ pod is up and running. Run the command below until the RabbitMQ pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
oc get pod -l app.kubernetes.io/instance=rabbitmq
oc wait pod -l app.kubernetes.io/instance=rabbitmq --for=condition=Ready --timeout=5m
oc get pod -l app.kubernetes.io/instance=rabbitmq
echo DONE

## Deploy Keycloak with helm. 

In [ ]:
OPENSHIFT_APPS_DOMAIN=apps.shift.nerc.mghpcc.org

helm upgrade --install keycloak oci://registry-1.docker.io/bitnamicharts/keycloak --version 21.7.1 \
  --set postgresql.enabled=false \
  --set-json 'resources={"requests": {"cpu": "200m", "memory": "500Mi"}}' \
  --set externalDatabase.host=postgresql \
  --set externalDatabase.user=keycloak \
  --set externalDatabase.password=$(oc get secret/postgres-pguser-keycloak -o jsonpath={.data.password} | base64 -d) \
  --set externalDatabase.database=keycloak \
  --set externalDatabase.port=5432 \
  --set ingress.enabled=true \
  --set ingress.hostname=ai-telemetry-keycloak.$OPENSHIFT_APPS_DOMAIN \
  --set adminIngress.hostname=ai-telemetry-keycloak.$OPENSHIFT_APPS_DOMAIN \
  --set-json 'extraEnvVars=[{"name":"KC_HOSTNAME_URL","value":"https://ai-telemetry-keycloak.'"$OPENSHIFT_APPS_DOMAIN"'"}, {"name":"KC_HOSTNAME_ADMIN_URL","value":"https://ai-telemetry-keycloak.'"$OPENSHIFT_APPS_DOMAIN"'"}]' \
  --set ingress.tls=true \
  --set ingress.selfSigned=false \
  --set production=true \
  --set tls.enabled=true \
  --set tls.autoGenerated=true \
  --set image.tag=24.0.5 \
  --set auth.existingSecret=keycloak-initial-admin \
  --set auth.adminUser=admin \
  --set auth.passwordSecretKey=admin-password
echo DONE

### View Keycloak pod details
After deploying the Helm Chart, it will take a minute before the Keycloak pod is up and running. Run the command below until the Keycloak pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
oc get pod/keycloak-0
oc wait pod/keycloak-0 --for=condition=Ready --timeout=2m
oc get pod/keycloak-0
echo DONE